# Árboles de Decisión
Actividad Lección 9 || Programación Python para Machine Learning

Objetivos:
* Conocer los principios en los que se basan los Árboles de decisión.
* Reconocer los componentes de un Árbol de decisión y entender cómo se construye.
* Dominar las técnicas de implementación de los Árboles de decisión en Python.
* Identificar los aspectos a tener en cuenta para mejorar el rendimiento de un Árbol de decisión.

Datos del alumno:
* Víctor Luque Martín
* Máster Avanzado en Programación en Python para Hacking, BigData y Machine Learning

Fecha: 27/12/2022

# Tabla de Contenidos
1. [Importes](#importes)
2. [Carga del dataset](#carga)
3. [Procesamiento](#procesamiento)
4. [Perceptrón Multicapa (MLP)](#mlp)
7. [Evaluación del modelo](#evaluacion)
8. [Optimizando MLP](#optimizando)

# Importes <a name="importes"></a>

In [1]:
import random, time
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection, metrics
from sklearn.tree import DecisionTreeClassifier, plot_tree

# Carga del dataset <a class="anchor" name="carga"></a>
Se cargan los datos del dataset “"

In [2]:
col_names = ["pelvic_incidence", "pelvic_tilt", "lumbar_lordosis_angle", 
             "sacral_slope", "pelvic_radius", "degree_spondylolisthesis", "class"]
df = pd.read_csv("column_3C.dat", sep=" ", header=None, names=col_names)
df.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.03,22.55,39.61,40.48,98.67,-0.25,DH
1,39.06,10.06,25.02,29.00,114.41,4.56,DH
2,68.83,22.22,50.09,46.61,105.99,-3.53,DH
3,69.30,24.65,44.31,44.64,101.87,11.21,DH
4,49.71,9.65,28.32,40.06,108.17,7.92,DH


# Análisis de los datos <a class="anchor" name="analisis"></a>
Comprobaremos si se trata de un dataset desbalanceado, ademas de comprobar si existen valores nulos.

In [3]:
df["class"].value_counts()

SL    150
NO    100
DH     60
Name: class, dtype: int64

In [4]:
df.isnull().sum()

pelvic_incidence            0
pelvic_tilt                 0
lumbar_lordosis_angle       0
sacral_slope                0
pelvic_radius               0
degree_spondylolisthesis    0
class                       0
dtype: int64

Como hemos podido observar, el dataset está desbalanceado, y no existen valores nulos, este primer dato es relevante a la hora de la generación del modelo.

# Procesamiento <a class="anchor" name="procesamiento"></a>
* Mapeo de las clases a números
* División de los datos de entrenamiento y prueba por sus variables dependientes e independientes

In [5]:
# Mapeo de la variable categórica "class"
df["class"] = df["class"].map({"DH":0, "SL":1, "NO":2})

# División X_train, y_train, X_test, y_test
seed = random.seed(time.time())
X_train, X_test, y_train, y_test = model_selection.train_test_split(df.drop("class", axis=1), 
                                                                    df["class"], 
                                                                    test_size=0.2, 
                                                                    random_state=seed)

# Árboles de Decisión <a class="anchor" name="dt"></a>
Los árboles de decisión son un tipo de algoritmo de aprendizaje supervisado que se utiliza para la clasificación y regresión. Los árboles de decisión son un tipo de modelo predictivo que se basa en la creación de un árbol de decisión a partir de los datos de entrenamiento. El árbol de decisión se construye a partir de un conjunto de reglas de decisión inferidas a partir de los datos de entrenamiento. Cada nodo interno del árbol representa un atributo, cada rama representa una regla de decisión y cada nodo hoja representa una clase o valor de predicción.

In [6]:
model = DecisionTreeClassifier(class_weight="balanced", max_depth=4)

# Evaluación del modelo <a class="anchor" name="evaluacion"></a>
Utilizando la validación cruzada K-Fold, se evaluará el rendimiento del modelo.

In [7]:
seed = random.seed(time.time())
kfold = model_selection.KFold(n_splits=5, random_state=seed, shuffle=True)
res_bal_acc = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring="balanced_accuracy")
res_acc = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring="accuracy")
df_res = pd.DataFrame({'KFold Balanced Accuracy Mean': res_bal_acc.mean(),
                       'KFold Balanced Accuracy Std': res_bal_acc.std(),
                       'KFold Accuracy Mean': res_acc.mean(),
                       'KFold Accuracy Std': res_acc.std()}, index=["Árbol de decisión"])
df_res

,KFold Balanced Accuracy Mean,KFold Balanced Accuracy Std,KFold Accuracy Mean,KFold Accuracy Std
Árbol de decisión,0.747808,0.028581,0.822857,0.0455


# Optimizando el Árbol de Decisión <a class="anchor" name="optimizando"></a>
Reutilizando la validación cruzada K-Fold, se emplarán diferentes valores de para ver cuál es el que mejor rendimiento tiene. Los parámetros para realizar la optimización son:
* **Criterio de división**: gini o entropía
* **Splitter**: mejor o aleatorio
* **Profundidad Máxima**: Del 4 al 10
* **Alphas**: 0, 0.001, 0.01, 0.1, 1
* **Máximo de características**: None, sqrt, log2 y del 5 al 10
* **Mínimo de muestras por hoja**: del 1 al 3
* **Mínimo de muestras para dividir**: del 2 al 4

Se generarán un total de 22680 modelos, y se seleccionará el que mejor rendimiento tenga.

In [8]:
criterias = ["gini", "entropy"]
splitters = ["best", "random"]
class_weights = ["balanced", None]
max_depth = [4, 5, 6, 7, 8, 9, 10]
ccp_alphas = [0.0, 0.001, 0.01, 0.1, 1.0]
max_features = [None, "sqrt", "log2", 5, 6, 7, 8, 9, 10]
min_samples_split = [2, 3, 4]
min_samples_leaf = [1, 2, 3]
df_metrics = pd.DataFrame(columns=["criteria", "splitter", "class_weight", 
                                   "max_depth", "ccp_alpha", "max_feature",
                                   "min_sample_split", "min_sample_leaf",
                                   "bacc", "baccy_err",
                                   "bacc_std", "acc", "acc_err", "acc_std"])

for criterion in criterias:
    for splitter in splitters:
        for class_weight in class_weights:
            for depth in max_depth:
                for ccp_alpha in ccp_alphas:
                    for max_feature in max_features:
                        for min_sample_split in min_samples_split:
                            for min_sample_leaf in min_samples_leaf:
                                model = DecisionTreeClassifier(criterion=criterion, 
                                                               splitter=splitter, 
                                                               class_weight=class_weight, 
                                                               max_depth=depth, 
                                                               ccp_alpha=ccp_alpha, 
                                                               max_features=max_feature, 
                                                               min_samples_split=min_sample_split, 
                                                               min_samples_leaf=min_sample_leaf)
                                res1 = model_selection.cross_val_score(model, X_train, y_train, 
                                                                       cv=kfold, scoring="balanced_accuracy")
                                res2 = model_selection.cross_val_score(model, X_train, y_train, 
                                                                       cv=kfold, scoring="accuracy")
                                df_metrics.loc[len(df_metrics)] = [
                                    criterion, splitter, class_weight, depth, ccp_alpha, 
                                    max_feature, min_sample_split, min_sample_leaf,
                                    res1.mean(), 1-res1.mean(), res1.std(), 
                                    res2.mean(), 1-res2.mean(), res2.std()
                                ]

print("Done Optimization!")

Done Optimization!


Una vez finalizado el proceso de optimización podemos siguiente observar el mejor resultado:

In [9]:
df_metrics.nsmallest(1, ['baccy_err', 'acc_err'])

,criteria,splitter,class_weight,max_depth,ccp_alpha,max_feature,min_sample_split,min_sample_leaf,bacc,baccy_err,bacc_std,acc,acc_err,acc_std
42,gini,best,balanced,4,0.0,6,4,1,0.839619,0.160381,0.051047,0.802204,0.197796,0.0462
